In [218]:
import numpy as np
import cv2 
import os

In [219]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [220]:
def feature_object_detection(template_img, template_gray, query_img, query_gray, min_match_number) :  #รับภาพต่างๆ มา min_match_number คือจำนวน matchที่น้อยที่สุดท้ายยอมรับได้ 
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)  #ทำการdetectAndCompute หาจุดเด่นของภาพ template  
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)  #ทำการหาคู่ match กันป้อนภาพ template_desc, query_desc เข้าไปสองภาพ k=2 เป็นคู่กัน
    good_matches = list()   
    good_matches_list = list()
    for m, n in matches :  #(matches[0])(match[1]) วนลูปไปเรื่อยๆจนจำนวน match ครบ
        if m.distance < 0.7*n.distance :  #เช็คความเหมือนว่าอยู่ในระยะที่ใกล้เคียงกันไหม
            good_matches.append(m)      #ถ้า m < n ที่ถูกคูณด้วย 0.7 เป็น good match ที่ดี
            good_matches_list.append([m])

    
    if len(good_matches) > min_match_number : # len เช็คจำนวนของ list มีจำนวนเท่าไหร่
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2) #วน for ตามจำนวน good matchแล้วนำค่า m มาหา queryIdx ค่าที่ได้จำป็นตำแหน่งของ template_kpt
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)  

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0) # H RANSAC #ป้อน src_pts, dst_pts ใช้ฟังก์ชัน RANSAC สามารถยอมรับที่ 10 pixcel
        # get the bounding box around template image
        h, w = template_img.shape[:2]  #ความสูง ความกว้างของ template_img เพื่อนำไปวาปปิ้งแล้วไปขีดเส้น
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)  #ตำแหน่ง template_img
        transformed_box = cv2.perspectiveTransform(template_box, H) #ทำการ วาปปิ้งภาพโดยใช้ perspectiveTransform ใส่ค่า template_box ลงไปจะเปลี่ยนแปลงไปตามค่า H จะให้ค่าตำแหน่งปัจจุบันของค่า H มา

        detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (255,0,0), 4, cv2.LINE_AA)  #วาดเส้น บน query_img 
        drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks) #วาดภาพ

        return detected_img, drawmatch_img
    else :
        print('Keypoints not enough')
        return query_img, query_img
    

In [221]:
vid = cv2.VideoCapture('010723305-main/videos/final_exam/Dataset-1/left_output-1.avi') #ทำการเพิ่มวิดีโอ อ่านวิดีโอ

template_img = cv2.imread('010723305-main/images/final_exam/Templates/Template-1.png')
template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)  #แปลงภาพให้เป็น gray scale 

while vid.isOpened() :
    ret, frame = vid.read() 
    
    if ret :
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        detected, drawmatch =  feature_object_detection(template_img, template_gray,frame,frame_gray, 8) #รับภาพมา นับจำนวนคู่ที่ แมท กัน 8 คู่
        
        cv2.imshow('Video frame', frame) #show รูป
        cv2.imshow('gray', frame_gray ) #show รูปที่เป็น gray scale

        if cv2.waitKey(int(1000/30)) & 0xFF == ord('k') : # this line control the period between image frame # 30 คือframe rate  #กำหนดเมื่อกด k จะออกจากวิดีโอ
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()

In [222]:
chessboard_size = (8,6) # Your chessboard size

# iterative termination criteria, maximum iterationm and epsilon
term_criteria = (cv2.TermCriteria_EPS+ cv2.TermCriteria_MAX_ITER, 30, 0.001)

In [223]:
# List to store object points and images point from all the images.
obj_points = list() # 3D points in heterogeneous Xi
img_points = list() # 2D points on image xi

# Defining the world coordinates for 3D points Xi
objp = np.zeros((1, chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
prev_img_shape = None

In [224]:
for image in images :
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(img_gray, chessboard_size, None) # xi in pixel (20,15)
    image_size = img_gray.shape
    # If found add obj points, image points afterthat refining them
    if ret == True :
        #print('Found')
        obj_points.append(objp) # Add Xi 3D
        
        corners2 = cv2.cornerSubPix(img_gray, corners, (11,11), (-1,-1), term_criteria) # Refining xi -> xi in subpixel, xi -> floating point (19.7, 15.1)
        img_points.append(corners2) # Add xi 2D

        #Draw and display the chessboard corners
        img = cv2.drawChessboardCorners(image, chessboard_size, corners2, ret)
        cv2.imshow('frame', img)
        cv2.waitKey(100)

cv2.destroyAllWindows()